In [ ]:
import pandas as pd

from quotaclimat.data_processing.factiva.explo_optimal_thresholds.utils_explo.llm_classif import (
    classify_dataframe_async,
    llm_gpt_4_1,
)

DATA_TEST_PATH = 'quotaclimat/data_processing/factiva/explo_optimal_thresholds/data_explo/factiva_data.xlsx'
SHEET_NAME = 'dataset'
EXPORT_PATH = 'quotaclimat/data_processing/factiva/explo_optimal_thresholds/data_explo/factiva_data_test_llm.xlsx'

In [ ]:
df = pd.read_excel(DATA_TEST_PATH, sheet_name=SHEET_NAME)
df['crises'] = None
df['maillons_par_crise'] = None
df['raw_json'] = None
df['text'] = df['title'] + ' ' + df['snippet'] + ' ' + df['body']

In [ ]:
out_df_4_1 = await classify_dataframe_async(
    df[df.raw_json.isna()].reset_index(drop=True)[[col for col in df.columns if col not in ['crises', 'maillons_par_crise', 'raw_json']]], text_col="text", llm=llm_gpt_4_1, batch_size=2
)

In [ ]:
valid_llm = out_df_4_1[out_df_4_1['raw_json'] != "{}"].copy()
valid_llm = (
    valid_llm[["an", "crises", "maillons_par_crise", "raw_json"]]
    .drop_duplicates(subset="an", keep="last")
    .set_index("an")
)

In [ ]:
for col in ["crises", "maillons_par_crise", "raw_json"]:
    mask = df[col].isna()
    df.loc[mask, col] = df.loc[mask, "an"].map(valid_llm[col])

In [ ]:
df[~df.raw_json.isna()].to_excel(EXPORT_PATH, sheet_name='test_dataset')